In [1]:
!unzip '/content/dataset5.zip'

Archive:  /content/dataset5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import numpy as np
import pandas as pd
import keras

In [3]:
import sklearn
from keras.models import Sequential
from keras.layers import Dense

In [4]:
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score



In [5]:
from nltk.corpus import stopwords



nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import tensorflow as tf

In [7]:
train_data = pd.read_csv(r"/content/train.csv")
test_data =  pd.read_csv(r"/content/train.csv")

In [8]:
train_data.head(3)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0


In [9]:
train_data['Text']=train_data['TITLE']+' '+train_data['ABSTRACT']
train_data.drop(columns=['TITLE','ABSTRACT'], inplace=True)
train_data.head(10)

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Text
0,1,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,2,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,3,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...
5,6,0,0,1,0,0,0,On maximizing the fundamental frequency of the...
6,7,0,1,0,0,0,0,On the rotation period and shape of the hyperb...
7,8,0,1,0,0,0,0,Adverse effects of polymer coating on heat tra...
8,9,0,1,0,0,0,0,SPH calculations of Mars-scale collisions: the...
9,10,0,0,0,0,1,0,$\mathcal{R}_{0}$ fails to predict the outbrea...


In [10]:
test_data['Text']=test_data['TITLE']+' '+test_data['ABSTRACT']
test_data.drop(columns=['TITLE','ABSTRACT'], inplace=True)
test_data.head(10)

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Text
0,1,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,2,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,3,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...
5,6,0,0,1,0,0,0,On maximizing the fundamental frequency of the...
6,7,0,1,0,0,0,0,On the rotation period and shape of the hyperb...
7,8,0,1,0,0,0,0,Adverse effects of polymer coating on heat tra...
8,9,0,1,0,0,0,0,SPH calculations of Mars-scale collisions: the...
9,10,0,0,0,0,1,0,$\mathcal{R}_{0}$ fails to predict the outbrea...


In [11]:
#Remove Stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

#Clean Text
def clean_text(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    return text

#stemming
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train_data['Text'] = train_data['Text'].apply(lambda x: remove_stopwords(x))
train_data['Text'] = train_data['Text'].apply(lambda x:clean_text(x))
train_data['Text'] = train_data['Text'].apply(stemming)


test_data['Text'] = test_data['Text'].apply(lambda x: remove_stopwords(x))
test_data['Text'] = test_data['Text'].apply(lambda x:clean_text(x))
test_data['Text'] = test_data['Text'].apply(stemming)

In [12]:
train_data.head(3)

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Text
0,1,1,0,0,0,0,0,reconstruct subject specif effect map predict ...
1,2,1,0,0,0,0,0,rotat invari neural network rotat invari trans...
2,3,0,0,1,0,0,0,spheric polyharmon poisson kernel polyharmon f...


In [13]:
columns = train_data.columns
print(columns)

Index(['ID', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'Text'],
      dtype='object')


In [14]:
columnsT = test_data.columns
print(columnsT)

Index(['ID', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'Text'],
      dtype='object')


In [15]:
X_train = train_data[columns[-1:]]
X_train.head(3)

,Text
0,reconstruct subject specif effect map predict ...
1,rotat invari neural network rotat invari trans...
2,spheric polyharmon poisson kernel polyharmon f...


In [16]:
print(np.shape(X_train))

(20972, 1)


In [17]:
y_train = train_data[columns[1:7]]
y_train.head(3)

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0


In [18]:
X_test = test_data[columnsT[-1:]]
X_test.head(3)

,Text
0,reconstruct subject specif effect map predict ...
1,rotat invari neural network rotat invari trans...
2,spheric polyharmon poisson kernel polyharmon f...


In [19]:
print(np.shape(X_test))

(20972, 1)


In [20]:
y_test = test_data[columnsT[1:7]]
y_test.head(3)

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0


In [21]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['Text'])

In [22]:
X_test_tfidf = tfidf_vectorizer.transform(X_test['Text'])

In [23]:
print(np.shape(X_train_tfidf))

(20972, 35401)


In [24]:
print(np.shape(X_test_tfidf))

(20972, 35401)


In [25]:
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.to_numpy()


In [26]:
print(np.shape(y_train))

(20972, 6)


In [27]:
if isinstance(y_test, pd.DataFrame):
    y_test = y_test.to_numpy()


In [28]:
print(np.shape(y_test))

(20972, 6)


In [29]:
# Define the neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dense(32, activation='relu'),
    Dense(6, activation='sigmoid')
])

In [30]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
              metrics=['accuracy'])

In [31]:
import scipy.sparse as sp

# Reorder the sparse matrix
X_train_tfidf_reordered = sp.csr_matrix(X_train_tfidf)
X_train_tfidf_reordered.sort_indices()


In [32]:
X_test_tfidf_reordered = sp.csr_matrix(X_test_tfidf)
X_test_tfidf_reordered.sort_indices()

In [33]:
print(np.shape(X_train_tfidf_reordered))

(20972, 35401)


In [34]:
print(np.shape(X_test_tfidf_reordered))

(20972, 35401)


In [35]:
# Train the model
history = model.fit(X_train_tfidf_reordered, y_train, epochs=50, batch_size=32)

Epoch 1/50
656/656 [==============================] - 48s 68ms/step - loss: 0.2720 - accuracy: 0.6969
Epoch 2/50
656/656 [==============================] - 12s 19ms/step - loss: 0.1456 - accuracy: 0.7892
Epoch 3/50
656/656 [==============================] - 7s 10ms/step - loss: 0.0996 - accuracy: 0.8278
Epoch 4/50
656/656 [==============================] - 6s 10ms/step - loss: 0.0648 - accuracy: 0.8477
Epoch 5/50
656/656 [==============================] - 5s 7ms/step - loss: 0.0390 - accuracy: 0.8609
Epoch 6/50
656/656 [==============================] - 5s 8ms/step - loss: 0.0216 - accuracy: 0.8704
Epoch 7/50
656/656 [==============================] - 4s 6ms/step - loss: 0.0109 - accuracy: 0.8775
Epoch 8/50
656/656 [==============================] - 5s 7ms/step - loss: 0.0051 - accuracy: 0.8786
Epoch 9/50
656/656 [==============================] - 4s 5ms/step - loss: 0.0024 - accuracy: 0.8842
Epoch 10/50
656/656 [==============================] - 5s 7ms/step - loss: 0.0012 - accuracy: 

In [37]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_tfidf_reordered, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

# Make predictions
y_pred = model.predict(X_test_tfidf_reordered)

656/656 [==============================] - 2s 3ms/step - loss: 1.6371e-07 - accuracy: 0.8867
Test Loss: 1.6370685784750094e-07
Test Accuracy: 0.8867061138153076
656/656 [==============================] - 1s 2ms/step
